# Census API - New York

In [1]:
# import packages

from us import states
import pandas as pd
import requests

# import doc with API key

import config


#### Census API Link References

In [2]:
# "https://api.census.gov/data/2019/acs/acs5/profile?get=NAME,DP05_0001E&for=state:36&key={0}".format(config.MY_API_KEY)
# "https://api.census.gov/data/2019/acs/acs5/profile?get=group(DP05)&for=school%20district%20(unified):29850&in=state:36&key={0}".format(config.MY_API_KEY)


## Get Table Names from Census API

In [3]:
# URL from Census API that lists all variables from American Community Survey as a JSON
# these are Census Data Profile tables - the most detailed tables available from the ACS

variableURL = "https://api.census.gov/data/2019/acs/acs5/variables"

In [4]:
# request method which will return data from URL

censusVarNames = requests.request("GET", variableURL)

In [5]:
# check to see if request method was successful by printing 6th object from JSON

print(censusVarNames.json()[5])

['B19001B_014E', 'Estimate!!Total:!!$100,000 to $124,999', 'HOUSEHOLD INCOME IN THE PAST 12 MONTHS (IN 2019 INFLATION-ADJUSTED DOLLARS) (BLACK OR AFRICAN AMERICAN ALONE HOUSEHOLDER)']


In [7]:
# JSON data can be displayed in a pandas dataframe
# JSON data from Census API has headers included

censusData = pd.DataFrame(columns=censusVarNames.json()[0], data=censusVarNames.json()[1:])
censusData = (censusData.sort_values(by=['name'], axis=0))

# you must reset the index so that the first item is 0 rather than the random numbers the dataframe assigns to each object
censusData.reset_index(drop = True, inplace = True)
censusData


,name,label,concept
0,AIANHH,Geography,None
1,AIHHTL,Geography,None
2,AIRES,Geography,None
3,ANRC,Geography,None
4,B01001A_001E,Estimate!!Total:,SEX BY AGE (WHITE ALONE)
...,...,...,...
27075,UA,Geography,None
27076,ZCTA,Geography,None
27077,for,Census API FIPS 'for' clause,Census API Geography Specification
27078,in,Census API FIPS 'in' clause,Census API Geography Specification


In [8]:
# filter table by name

def filterByTableName(string):
    x = censusData[censusData['name'].str.contains(string)]
    return x

# filterByTableName('B1')


In [9]:
# filter table by label

def filterByLabel(string):
    x = censusData[censusData['label'].str.contains(string)]
    return x

# filterByLabel('Median income')

In [83]:
# fitler table by list of table codes

def filterByTableName(tableList):
    df = censusData[censusData['name'].isin(tableList)]
    df.reset_index(drop = True, inplace = True)
    return df

## Import CSV

#### If a user is interested in looking at only select parts of the entire US Census set, save a separate text file with your list. 
#### You can pick and choose which school districts after compiling all the data, but it helps to filter out the data earlier on to minimize the size of your data frame

In [10]:
# Returns the dataframe of school districts

def schoolDistrictCSV(CSVFile):
    schoolDistrict = pd.read_csv(CSVFile, names=['School District Name'])
    return schoolDistrict

#### GEOIDs are going to be critical to using the Census API.

Create a new Dataframe, which combines all rows based on their "School District Name."
This will allow that first layer of filtering to happen -> you get the GEOIDs for all of the school districts you are interested in looking at.

Need to further investigate and understand Pandas concat, merge, join. Not sure why merge worked - so need to dig into that.


In [11]:
# Read Census data set which lists GEOIDs per each school district in state
# For list of other states and other GEOID data sets, see URL https://www2.census.gov/geo/docs/reference/codes/files/

def schoolDistrictGEOID(URL):
    df = pd.read_csv(URL,names=['State', 'ID','GEOID','School District Name', 'Class'])
    return df

In [12]:
# Function that merges user CSV data frame with the GEOID data frame into a df

def mergedSchoolDist(df1, df2):
    CountiesGEOID = df1.merge(df2, on=['School District Name'],how='inner')
    return CountiesGEOID

In [56]:
# Get all GEOIDs in a single list
# Make all GEOIDs into a string only
# Need to be a single strin in order to run Census API call

def GEOIDonly(dataFrame):
    x = dataFrame['GEOID'].tolist()
    separator = ', '
    GEOIDsinglestring = separator.join(map(str, x))
    return GEOIDsinglestring

In [32]:
#Function to turn list of strings into a single string

def GEOIDonlyx(dataFrame):
    z = dataFrame['GEOID'].tolist()
    separator = ', '
    print(z)

In [16]:
# Test with Nassau County School Districts

selectedNYCounties = schoolDistrictCSV((r'Data\USCensusNYSchDist.csv'))

In [17]:
# Test with Nassau County School Districts

NYURL = ("https://www2.census.gov/geo/docs/reference/codes/files/st36_ny_schdist.txt")
NY = schoolDistrictGEOID(NYURL)

In [57]:
# Test merging user input school district CSV with census GEOID list

NYCountiesGEOID = mergedSchoolDist(NY, selectedNYCounties)

In [58]:
# Test grab GEOIDs only

NYGEOID = (GEOIDonly(NYCountiesGEOID))

print(type(NYGEOID))

<class 'str'>


In [84]:
# Test filter by list of known table names

xxx = ['B01001_002E']

filterByTableName(xxx)

,name,label,concept
0,B01001_002E,Estimate!!Total:!!Male:,SEX BY AGE


## Refactoring into Formulas

#### Breakdown into small discrete formulas

In [44]:
def jsontodf(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [45]:
def dataProfSchDist(censusVariables,schoolDistricts):
    # first URL is for the detailed table. second URL is for the data profiles.
    # URL = "https://api.census.gov/data/2019/acs/acs5?get=NAME,{0}&for=school%20district%20(unified):{1}&in=state:36&key={2}".format(censusVariables,schoolDistricts,config.MY_API_KEY)

    URL = "https://api.census.gov/data/2019/acs/acs5/profile?get=NAME,{0}&for=school%20district%20(unified):{1}&in=state:36&key={2}".format(censusVariables,schoolDistricts,config.MY_API_KEY)
    return requests.request("GET", URL)

In [46]:
def deTabSchDist(censusVariables,schoolDistricts):
    # first URL is for the detailed table. second URL is for the data profiles.
    # URL = "https://api.census.gov/data/2019/acs/acs5?get=NAME,{0}&for=school%20district%20(unified):{1}&in=state:36&key={2}".format(censusVariables,schoolDistricts,config.MY_API_KEY)

    URL = "https://api.census.gov/data/2019/acs/acs5/?get=NAME,{0}&for=school%20district%20(unified):{1}&in=state:36&key={2}".format(censusVariables,schoolDistricts,config.MY_API_KEY)
    return requests.request("GET", URL)

#### Mega Formula

In [47]:
def dataProfile(cenVar, schDisID):
    x = dataProfSchDist(cenVar,schDisID)
    return jsontodf(x)

def detailedTable(cenVar, schDisID):
    x = deTabSchDist(cenVar,schDisID)
    return jsontodf(x)

#### Test

In [60]:
# Getting Census Data: Data Profiles

selectedVar = 'DP05_0002E,DP05_0003E'

dataProfile(selectedVar,NYGEOID)

,NAME,DP05_0002E,DP05_0003E,state,school district (unified)
0,"Roslyn Union Free School District, New York",8837,9761,36,25050
1,"East Williston Union Free School District, New...",4217,4669,36,10050
2,"North Shore Central School District, New York",8408,8240,36,26370
3,"Garden City Union Free School District, New York",10806,11693,36,11760
4,"Wantagh Union Free School District, New York",8279,8460,36,29850
5,"Manhasset Union Free School District, New York",8015,8832,36,18270
6,"Port Washington Union Free School District, Ne...",15199,15829,36,23580
7,"Rockville Centre Union Free School District, N...",10490,11579,36,24780
8,"Hewlett-Woodmere Union Free School District, N...",9486,10591,36,31710
9,"Levittown Union Free School District, New York",22308,23358,36,17160


In [61]:
# Getting Census Data: Detailed Tables

selectedVar = 'B01001_002E,B01001_026E,B19013_001E'

detailedTable(selectedVar,NYGEOID)

,NAME,B01001_002E,B01001_026E,B19013_001E,state,school district (unified)
0,"Roslyn Union Free School District, New York",8837,9761,160098,36,25050
1,"East Williston Union Free School District, New...",4217,4669,177868,36,10050
2,"North Shore Central School District, New York",8408,8240,135563,36,26370
3,"Garden City Union Free School District, New York",10806,11693,174886,36,11760
4,"Wantagh Union Free School District, New York",8279,8460,155588,36,29850
5,"Manhasset Union Free School District, New York",8015,8832,207198,36,18270
6,"Port Washington Union Free School District, Ne...",15199,15829,134627,36,23580
7,"Rockville Centre Union Free School District, N...",10490,11579,108797,36,24780
8,"Hewlett-Woodmere Union Free School District, N...",9486,10591,128547,36,31710
9,"Levittown Union Free School District, New York",22308,23358,126665,36,17160


In [88]:
# Population by Age, Gender, and Demographic

Pop = 11255,4,5,20,36,51,67,82,98,113,129,144,160,175,191,206,222,237,253,268


filterdf = censusData[censusData.index.isin(Pop)]
x = filterdf['name'].tolist()
x

['B01001A_001E',
 'B01001A_002E',
 'B01001A_017E',
 'B01001B_002E',
 'B01001B_017E',
 'B01001C_002E',
 'B01001C_017E',
 'B01001D_002E',
 'B01001D_017E',
 'B01001E_002E',
 'B01001E_017E',
 'B01001F_002E',
 'B01001F_017E',
 'B01001G_002E',
 'B01001G_017E',
 'B01001H_002E',
 'B01001H_017E',
 'B01001I_002E',
 'B01001I_017E',
 'B19013A_001E']

In [89]:
s = singleString(x)
print(s)

B01001A_001E,B01001A_002E,B01001A_017E,B01001B_002E,B01001B_017E,B01001C_002E,B01001C_017E,B01001D_002E,B01001D_017E,B01001E_002E,B01001E_017E,B01001F_002E,B01001F_017E,B01001G_002E,B01001G_017E,B01001H_002E,B01001H_017E,B01001I_002E,B01001I_017E,B19013A_001E


In [93]:
testtest = detailedTable(s,NYGEOID)
testtest


testtest[x] = testtest[x].astype(int)



SyntaxError: invalid syntax (<ipython-input-93-b7d7931e1d2a>, line 5)

In [92]:
# Filter by integer value
newDf = testtest.loc[(testtest['B01001A_001E']>40000)]
newDf

TypeError: '>' not supported between instances of 'str' and 'int'

## Matplotlib

In [ ]:
from matplotlib import pyplot as plt
from matplotlib.ticker import MultipleLocator
import numpy as np

In [ ]:
allDistricts = newDf["NAME"].tolist()
totalPop = newDf["B01001A_001E"].tolist()

In [ ]:
fig = plt.figure(figsize=(50,50))

plt.xticks(fontsize=50)
plt.yticks(fontsize=50)


plt.title('State Population',fontsize=50)
plt.ylabel('allDistricts',fontsize=50)
plt.xlabel('Population',fontsize=50)

plt.barh(allDistricts, totalPop, 0.8, color = "#444444",label="TotalPopulation")
#plt.xticks([0,10])

plt.legend(fontsize=50)
plt.show()